In [1]:
#import openpyxl
#openpyxl.__version__

import sqlite3
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

file = 'data/SampleData.xlsx'
output = 'data/output.xlsx'

## sqlite3
engine = create_engine('sqlite://', echo=False)   # just keep it in RAM, not to store into database
# engine = create_engine('sqlite:///store.db', echo=False)   # it'll create 'store.db'

df = pd.read_excel(file, sheet_name='SalesOrders')
#print(df)

# To keep only date part when using pandas.to_datetime
df['OrderDate'] = df['OrderDate'].dt.date
#                                                          .apply(lambda x: dt.datetime.strptime(x,'%d%b%Y.%f'))

# put it in my temporary data file 'salesorders'
df.to_sql(name='salesorders', con=engine, if_exists='replace', index=False)

# results = engine.execute("SELECT * FROM salesorders")
# results = engine.execute("SELECT * FROM salesorders WHERE Region='East' \
#                                             AND Units>50")
results = engine.execute("SELECT * FROM salesorders WHERE Region in('East', 'West') \
                                            AND Units>50")
final = pd.DataFrame(results, columns=df.columns)
# final.to_excel(output, index=False)

final

,OrderDate,Region,Rep,Item,Units,Unit Cost,Total
0,2018-01-06,East,Jones,Pencil,95,1.99,189.05
1,2018-03-15,West,Sorvino,Pencil,56,2.99,167.44
2,2018-04-01,East,Jones,Binder,60,4.99,299.40
3,2018-06-08,East,Jones,Binder,60,8.99,539.40
4,2018-07-29,East,Parent,Binder,81,19.99,1619.19
5,2018-10-22,East,Jones,Pen,64,8.99,575.36
6,2018-12-29,East,Parent,Pen Set,74,15.99,1183.26
7,2019-04-27,East,Howard,Pen,96,4.99,479.04
8,2019-07-04,East,Jones,Pen Set,62,4.99,309.38
9,2019-09-27,West,Sorvino,Pen,76,1.99,151.24


In [2]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import os

user = 'root'
passw = 'nbuser'
host =  'localhost'  # either localhost or ip e.g. '172.17.0.2' or hostname address 
port = 3306 
database = 'emp_db'

engine = create_engine('mysql+pymysql://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database , echo=False)

results = engine.execute("SELECT * FROM employee WHERE salary > 55000")
print(results.fetchall())



################

directory = r'directoryLocation'  # path of csv file
csvFileName = 'data.csv'

# df = pd.read_csv(os.path.join(directory, csvFileName ))
df = pd.read_csv(os.path.join(csvFileName ))
# print(df)

df.to_sql(name=csvFileName[:-4], con=engine, if_exists = 'replace', index=False)

# if_exists: {'fail', 'replace', 'append'}, default 'fail'
#      fail: If table exists, do nothing.
#      replace: If table exists, drop it, recreate it, and insert data.
#      append: If table exists, insert data. Create if does not exist.



[(2002, 'Supe', 'Man', 'M', 1, 75000, 1, datetime.date(1999, 8, 19)), (2005, 'John', 'Park', 'M', 2, 65000, 3, datetime.date(1960, 11, 23)), (2006, 'Kimm', 'Lee', 'F', 6, 90000, 2, datetime.date(1970, 6, 5)), (2007, 'Say', 'Yes', 'M', 1, 70000, 1, datetime.date(1990, 2, 21)), (2008, 'JunHo', 'Lee', 'M', 3, 65000, 2, datetime.date(1980, 3, 5)), (2009, 'GwangHo', 'Kim', 'M', 5, 65000, 3, datetime.date(1990, 2, 21))]


In [7]:
import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font

wb = openpyxl.load_workbook('data/Sample_data.xlsx')

sheet = wb["Sheet1"]

for col in sheet.columns:
    max_length = 0
    column = col[0].column # Get the column number
    col_name = get_column_letter(column)
#     print(column)
    
    for cell in col:
        if cell.coordinate in sheet.merged_cells: # not check merge_cells
            continue
        try: # Necessary to avoid error on empty cells
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
#                 print(cell.value)
        except:
            pass
        
    adjusted_width = (max_length + 2) * 1.2
    sheet.column_dimensions[col_name].width = adjusted_width


column_widths = []
for row in sheet.iter_rows():
    for i, cell in enumerate(row):
        try:
            column_widths[i] = max(column_widths[i], len(str(cell.value)))
        except IndexError:
            column_widths.append(len(str(cell.value)))

for i, column_width in enumerate(column_widths):
    sheet.column_dimensions[get_column_letter(i + 1)].width = column_width
    
print(column_widths)


[19, 10, 14, 35, 41, 22, 12, 17, 12, 12]


D:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:296: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [8]:
import openpyxl
#openpyxl.__version__

# import os
# os.chdir('C:/Users/Tiger KT/jupyter-notebook')

def as_text(value):
    if value is None:
        return ""
    return str(value)

wb  = openpyxl.load_workbook('data/Sample_data.xlsx')
# type(wb)

# wb.get_sheet_names()
wb.sheetnames

# sheet = wb.get_sheet_by_name('Sheet1')
ws = wb['Sheet1']


def adjust_column_width_from_col(ws, min_row, min_col, max_col):

        column_widths = []

        for i, col in enumerate(
                    ws.iter_cols(min_col=min_col, max_col=max_col, min_row=min_row)
                ):

            for cell in col:
                value = cell.value
                if value is not None:
#                     print(value)
                    
                    if isinstance(value, str) is False:
                        value = str(value)

                    try:
                        column_widths[i] = max(column_widths[i], len(value))
                    except IndexError:
                        column_widths.append(len(value))

#         print(column_widths)
        
        for i, width in enumerate(column_widths):

            col_name = get_column_letter(min_col + i)
            value = column_widths[i] + 2
            ws.column_dimensions[col_name].width = value
            print(col_name, value)
            
adjust_column_width_from_col(ws, 1,1, ws.max_column)

A 21
B 12
C 16
D 37
E 43
F 24
G 14
H 19
I 14
J 14
